In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

In [2]:
from platform import python_version
import sys
print('Python: ' + python_version()) # Python: 3.12.2
print('pandas: ' + pd.__version__) # pandas: 2.2.1
print ('plotly: ' + sys.modules["plotly"].__version__) # plotly: 5.22.0
print('dash_bootstrap_components: ' + dbc.__version__) # dash_bootstrap_components: 1.6.0

Python: 3.12.2
pandas: 2.2.1
plotly: 5.22.0
dash_bootstrap_components: 1.6.0


In [3]:
FireRP_MWIR = pd.read_csv("./data/FireRP_dataset_MWIR.csv", index_col = 0)

In [4]:
global_figure_styles = dict(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor= '#e4edf4')

In [5]:
# Create a Dash application instance
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    # Header for the application
    html.H1(children='Data Visualisation FireRF', style={'textAlign':'center'}),
    # Radio buttons to select time aggregation (Daily or Monthly)
    dcc.RadioItems(
        id='Time Aggregation', 
        options=["Daily", "Monthly"],
        value="Daily",
        inline=True
    ),
    # Slider to select the month
    dcc.Slider(
        0, 11, step=None,
        marks={
            0: '2020-03',
            1: '2020-04',
            2: '2020-05',
            3: '2020-06',
            4: '2020-07',
            5: '2020-08',
            6: '2020-09',
            7: '2020-10',
            8: '2020-11',
            9: '2020-12',
            10: '2021-01',
            11: '2021-02',
        },
        value = 0,
        id='Month',
        included=False
    ),
    # Fade component to control the visibility of the day slider
    dbc.Fade(
        dcc.Slider(
            min = 1,
            max = 31,
            step = 1,
            value = 1,
            id='Day',
            included=False
        ),
        id="fade",
        is_in=True,
    ),
    # Graph component to display the plot
    dcc.Graph(id="graph"),
]),

# Callback to toggle the visibility of the day slider based on time aggregation
@callback(
    Output("fade", "is_in"),
    Input('Time Aggregation', 'value'),
)
def toggle_fade(timeAgg):
    # Hide the day slider if time aggregation is monthly
    if timeAgg == "Monthly":
        return False
    else:
        return True

# Callback to set the maximum day value based on the selected month
@callback(
    Output('Day', 'max'),
    Input('Month', 'value')
)
def set_day_value(month):
    # Match the month value to return the corresponding maximum number of days
    match month:
        case 0: return 31
        case 1: return 30
        case 2: return 31
        case 3: return 30
        case 4: return 31
        case 5: return 31
        case 6: return 30
        case 7: return 31
        case 8: return 30
        case 9: return 31
        case 10: return 31
        case 11: return 27
        case _: return 31

# Callback to update the plot based on the selected time aggregation, month, and day
@callback(
    Output("graph", "figure"), 
    Input('Time Aggregation', 'value'),
    Input('Month', 'value'),
    Input('Day', 'value')
)
def display_choropleth(timeAgg, month, day):
    # Adjust the month value for indexing
    month = (month+3)%13 + int(month/10)
    if timeAgg == 'Monthly':
        # Create a density map for the selected month
        fig = px.density_mapbox(FireRP_MWIR.loc[(FireRP_MWIR.loc[:,'Month'] == month),:], 
                                lat='Latitude', 
                                lon='Longitude', 
                                z='FRP_MWIR', 
                                radius=3,
                                center=dict(lat=25, lon=0), zoom=1,
                                mapbox_style="open-street-map")
        fig.update_layout(
            **global_figure_styles
        )
    else:
        # Create a scattergeo plot for the selected day       
        fig = go.Figure(data=go.Scattergeo(
            lat = FireRP_MWIR.loc[(FireRP_MWIR.loc[:,'Month'] == month) & (FireRP_MWIR.loc[:,'Day'] == day), 'Latitude'],
            lon = FireRP_MWIR.loc[(FireRP_MWIR.loc[:,'Month'] == month) & (FireRP_MWIR.loc[:,'Day'] == day), 'Longitude'],
            text = "FRP: " + FireRP_MWIR.loc[(FireRP_MWIR.loc[:,'Month'] == month) & (FireRP_MWIR.loc[:,'Day'] == day), 'FRP_MWIR'].astype(str) ,
            marker = dict(
                color = FireRP_MWIR.loc[(FireRP_MWIR.loc[:,'Month'] == month) & (FireRP_MWIR.loc[:,'Day'] == day), 'log_FRP_MWIR'],
                colorscale = 'Reds',
                opacity = 0.7,
            
            )
        ))
        fig.update_layout(
            geo_scope='world', **global_figure_styles
        )
    return fig


In [6]:
if __name__ == '__main__':
    app.run(debug=True, port = 8060)